### spark session and libraries

In [11]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions
from pyspark.sql.functions import col, udf
from IPython.display import display
from datetime import datetime
from pyspark.sql.types import FloatType
from time import mktime
from datetime import datetime
    
spark_conf = SparkConf() \
    .setAll([
         ['spark.serializer','org.apache.spark.serializer.KryoSerializer'],
         ['spark.rdd.compress','true'],   
    ])

spark = SparkSession \
    .builder \
    .appName("pager") \
    .config(conf=spark_conf) \
    .enableHiveSupport() \
    .getOrCreate()

## create dataframe and drop low level features

In [15]:
df = spark.read.csv('hdfs://ip-172-31-51-46.ec2.internal/pager/sample.csv', header=True,
                   inferSchema=True, ignoreLeadingWhiteSpace=None, ignoreTrailingWhiteSpace=None)


time_difference_func = udf(lambda x, y: (datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p') - 
                                         datetime.strptime(y, '%m/%d/%Y %I:%M:%S %p')).total_seconds()/3600, 
                           FloatType())

df = df.dropna(how='any', subset=['Created Date','Closed Date'])

df = df.withColumn('Time Taken', time_difference_func(df['Closed Date'], df['Created Date']))

df = df[['Unique Key', 'Created Date','Closed Date', 'Time Taken', 'Agency', 'Complaint Type', 'Location Type',
         'Incident zip', 'Address Type', 'City', 'Facility Type', 'Borough', 'Status', 'School Name']]

## exploratory analysis

In [41]:
grouped_counts_dict = {}
import pandas

for col in ['Agency', 'Complaint Type', 'Location Type', 'Incident zip', 'Address Type', 'City',
               'Facility Type', 'Borough', 'Status', 'School Name']:
    group = [col] + ['Time Taken']
    grouped = df.select(group).groupBy(col)
    grouped_counts_dict[col] = {}
    grouped_counts_dict[col]['counts'] = grouped.count()
    grouped_counts_dict[col]['mean_time_taken'] = grouped.mean('Time Taken')
    grouped_counts_dict[col]['stdddev_time_taken'] = grouped.agg(functions.stddev('Time Taken'))
    
for col in ['Agency', 'Complaint Type', 'Location Type', 'Incident zip', 'Address Type', 'City',
               'Facility Type', 'Borough', 'Status', 'School Name']:
    display(grouped_counts_dict[col]['counts'].toPandas())

ImportError: No module named 'pandas'

In [27]:
df[['Agency', 'Time Taken']].show()

+------+----------+
|Agency|Time Taken|
+------+----------+
|  NYPD| 4.3283334|
|  NYPD|      1.69|
|  NYPD|     0.465|
|  NYPD| 1.0588889|
|   DPR|  287.7514|
|  DSNY| 2517.3303|
|   DOT|  26.38861|
|   HPD|   218.425|
|   DOF| 147.82222|
|   DHS| 1.6533333|
|  NYPD|0.31638888|
|   DOT| 1.0166667|
|   DOT|      1.95|
|   DEP| 29.916666|
|   HPD|  176.7886|
|  NYPD|     0.965|
|  NYPD| 7.0372224|
|  NYPD| 4.6738887|
|  NYPD| 1.0911111|
|  DSNY|  683.1519|
+------+----------+
only showing top 20 rows



In [ ]:
print()

In [40]:
means_grouped_by_cols_dict = {}

for col in df.columns:
    means_grouped_by_cols_dict[col] = df[[col, 'Time Taken']].groupBy[col].mean()

['Unique Key',
 'Created Date',
 'Closed Date',
 'Agency',
 'Agency Name',
 'Complaint Type',
 'Descriptor',
 'Location Type',
 'Incident Zip',
 'Incident Address',
 'Street Name',
 'Cross Street 1',
 'Cross Street 2',
 'Intersection Street 1',
 'Intersection Street 2',
 'Address Type',
 'City',
 'Landmark',
 'Facility Type',
 'Status',
 'Due Date',
 'Resolution Description',
 'Resolution Action Updated Date',
 'Community Board',
 'Borough',
 'X Coordinate (State Plane)',
 'Y Coordinate (State Plane)',
 'Park Facility Name',
 'Park Borough',
 'School Name',
 'School Number',
 'School Region',
 'School Code',
 'School Phone Number',
 'School Address',
 'School City',
 'School State',
 'School Zip',
 'School Not Found',
 'School or Citywide Complaint',
 'Vehicle Type',
 'Taxi Company Borough',
 'Taxi Pick Up Location',
 'Bridge Highway Name',
 'Bridge Highway Direction',
 'Road Ramp',
 'Bridge Highway Segment',
 'Garage Lot Name',
 'Ferry Direction',
 'Ferry Terminal Name',
 'Latitude',


In [3]:
df[['Agency', 'Time Taken']].groupBy(['Agency']).mean().show()

Py4JJavaError: An error occurred while calling o430.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 5, ip-172-31-59-33.ec2.internal, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/mnt/yarn/usercache/hadoop/appcache/application_1512765222948_0011/container_1512765222948_0011_01_000002/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/mnt/yarn/usercache/hadoop/appcache/application_1512765222948_0011/container_1512765222948_0011_01_000002/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/mnt/yarn/usercache/hadoop/appcache/application_1512765222948_0011/container_1512765222948_0011_01_000002/pyspark.zip/pyspark/serializers.py", line 220, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
 

## data aggreation

In [5]:
df.count()

16610832

In [6]:
df[df['Created Date'] > df['Closed Date']].count()

1587954

In [36]:
import datetime

a = '2014-05-06 12:00:56'
b = '2013-03-06 16:08:22'

start = datetime.datetime.strptime(a, '%Y-%m-%d %H:%M:%S')
ends = datetime.datetime.strptime(b, '%Y-%m-%d %H:%M:%S')

diff = (start - ends)
diff.days

425